In [ ]:
# ============================================================
# INSTALL DEPENDENCIES
# ============================================================
!pip install -q langgraph reportlab

# ============================================================
# IMPORTS
# ============================================================
import random, uuid
from datetime import datetime
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4

# ============================================================
# INVENTORY
# ============================================================
INVENTORY = {
    "shirts": [
           {

        "name": "Blue Formal Shirt",
        "stock": 5,
        "price": 1999,
        "sizes": ["S", "M", "L", "XL"]
    },
    {

        "name": "Black Formal Shirt",
        "stock": 3,
        "price": 1799,
        "sizes": ["S", "M", "L", "XL"]
    },
    {

        "name": "White Office Shirt",
        "stock": 5,
        "price": 2199,
        "sizes": ["M", "L", "XL"]
    },
    ],
    "pants": [{

        "name": "Black Slim Fit Pant",
        "stock": 5,
        "price": 2499,
        "sizes": ["30", "32", "34", "36"]
    },
    {

        "name": "Navy Blue Chinos",
        "stock": 5,
        "price": 2299,
        "sizes": ["30", "32", "34", "36"]
    },
    ],
    "ethnic": [
        {"name": "Ethnic Wear 1", "stock": 2, "price": 3499},
    ],
    "athleisure": [
        {"name": "Athleisure Wear 1", "stock": 7, "price": 1599},
    ]
}

COMPLEMENTARY = {
    "shirts": ["pants"],
    "pants": ["shirts"],
    "ethnic": ["ethnic"],
    "athleisure": ["athleisure"]
}

# ============================================================
# HELPERS
# ============================================================
def norm(t): return t.strip().lower()
def available_products(cat): return [i["name"] for i in INVENTORY[cat] if i["stock"] > 0]

def smart_recommend(category, cart):
    avg_price = sum(i["price"] for i in cart) / len(cart) if cart else 2000
    in_cart = {i["product"] for i in cart}
    recs = []
    for cat in COMPLEMENTARY[category]:
        for i in INVENTORY[cat]:
            if i["name"] not in in_cart:
                score = 1 / (1 + abs(i["price"] - avg_price))
                recs.append((i["name"], score))
    return [r[0] for r in sorted(recs, key=lambda x: x[1], reverse=True)[:1]]

# ============================================================
# STATE
# ============================================================
class FashionState(TypedDict):
    user_input: str
    step: str
    category: str
    product: str
    size: str
    cart: list
    cart_total: int
    discount: int
    final_price: int
    payment_attempts: int
    recommended_items: list
    exchange_item: dict
    response: str

# ============================================================
# INVOICE
# ============================================================
def generate_invoice_pdf(state):
    file = f"invoice_{uuid.uuid4().hex[:6]}.pdf"
    doc = SimpleDocTemplate(file, pagesize=A4)
    styles = getSampleStyleSheet()
    data = [["Product", "Size", "Price"]]
    for i in state["cart"]:
        data.append([i["product"], i["size"], f"₹{i['price']}"])
    data += [["", "Total", f"₹{state['final_price']}"]]
    doc.build([
        Paragraph("SMART FASHION STORE", styles["Title"]),
        Spacer(1, 10),
        Table(data)
    ])
    return file

# ============================================================
# AGENTS
# ============================================================
def ask_product(s):
    text = norm(s["user_input"])
    for c in INVENTORY:
        if c in text or text in c:   # ◀️ FIX
            s["category"] = c
            s["response"] = f"Available {c}:\n" + ", ".join(available_products(c))
            s["step"] = "select_product"
            return s
    s["response"] = "What are you shopping for? (shirts / pants / ethnic / athleisure)"
    return s


def select_product(s):
    user_input = norm(s["user_input"])
    category_products_data = INVENTORY[s["category"]]

    selected_product_item = None

    # Try to match by full name first
    for item_data in category_products_data:
        if norm(item_data["name"]) == user_input:
            selected_product_item = item_data
            break

    # If not found by full name, try to match by number (1-based index)
    if not selected_product_item:
        try:
            product_index = int(user_input) - 1
            available_names = available_products(s["category"])
            if 0 <= product_index < len(available_names):
                # Find the actual product dictionary from INVENTORY using the name
                product_name = available_names[product_index]
                selected_product_item = next(item_data for item_data in category_products_data if item_data["name"] == product_name)
        except ValueError:
            pass # Not a number, continue to error handling

    if selected_product_item:
        s["product"] = selected_product_item["name"]
        s["price"] = selected_product_item["price"]
        s["response"] = "Select size (S/M/L/XL)"
        s["step"] = "select_size"
    else:
        # If no product was found, provide a user-friendly error message and allow re-entry
        available_names = available_products(s['category'])
        if available_names:
            s["response"] = f"Sorry, I couldn't find '{s['user_input']}'. Please choose from {', '.join(available_names)}. You can also use the corresponding number (e.g., '1' for {available_names[0]})."
        else:
            s["response"] = f"Sorry, there are no products available in the {s['category']} category right now."
        s["step"] = "select_product" # Stay in the same step to ask again
    return s

def select_size(s):
    s["size"] = s["user_input"].upper()
    s["response"] = "Add to cart? (yes/no)"
    s["step"] = "cart_decision"
    return s

def cart_decision(s):
    if norm(s["user_input"]) == "yes":
        s["cart"].append({"product": s["product"], "size": s["size"], "price": s["price"]})
        s["cart_total"] += s["price"]
        rec = smart_recommend(s["category"], s["cart"])
        if rec:
            s["recommended_items"] = rec
            s["response"] = f"Recommended: {rec[0]}. Add? (yes/no)"
            s["step"] = "recommendation"
            return s
    s["response"] = "Shop more? (yes/no)"
    s["step"] = "shop_more"
    return s

def recommendation(s):
    if norm(s["user_input"]) == "yes":
        r = s["recommended_items"][0]
        price = next(i["price"] for c in INVENTORY for i in INVENTORY[c] if i["name"] == r)
        s["cart"].append({"product": r, "size": "M", "price": price})
        s["cart_total"] += price
    s["response"] = "Shop more? (yes/no)"
    s["step"] = "shop_more"
    return s

def shop_more(s):
    if norm(s["user_input"]) == "yes":
        s["step"] = "ask_product"
        s["response"] = "What would you like next?"
        return s

    summary = "\n".join([f"{i['product']} – ₹{i['price']}" for i in s["cart"]])
    s["response"] = (
        "🛒 CART SUMMARY:\n" + summary +
        f"\n\nSubtotal: ₹{s['cart_total']}\n\n"
        "Offers:\n1) HDFC ₹300\n2) ICICI ₹250\n3) SBI ₹200\nChoose offer:"
    )
    s["step"] = "apply_offer"
    return s

def apply_offer(s):
    offers = {"1": 300, "2": 250, "3": 200}
    s["discount"] = offers.get(s["user_input"], 0)
    s["final_price"] = s["cart_total"] - s["discount"]
    s["response"] = f"Final amount: ₹{s['final_price']}\nBuy online or store?"
    s["step"] = "payment"
    return s

def payment(s):
    choice = s["user_input"].lower()

    if choice == "store":
        s["response"] = "Please complete payment at the nearest store."
        s["step"] = "support"
        return s

    # ONLINE PAYMENT
    s["payment_attempts"] += 1
    success = random.random() < 0.7

    if success:
        s["response"] = "✅ Payment successful! Delivery in 3–5 days."
        s["step"] = "support"
        return s

    if s["payment_attempts"] == 1:
        s["response"] = "❌ Payment failed. Retry? (yes / no)"
        s["step"] = "payment"
        return s

    s["response"] = "❌ Payment failed twice. Please pay at store."
    s["step"] = "support"
    return s


def support(s):
    invoice_text, filename = generate_invoice(s)

    s["response"] = (
        "✅ Order completed successfully!\n\n"
        "🧾 INVOICE:\n"
        f"{invoice_text}\n"
        f"📁 Invoice saved as: {filename}\n\n"

    )


    s["response"] = f"Order complete. Invoice: {invoice_text}\nNeed return/exchange? (yes/no)"
    s["step"] = "post_support"
    return s

def generate_invoice(s):
    invoice_id = str(uuid.uuid4())[:8]
    date = datetime.now().strftime("%d-%m-%Y %H:%M")

    items = "\n".join(
        [f"{i+1}. {item['product']} (Size {item['size']}) - ₹{item['price']}"
         for i, item in enumerate(s["cart"])]
    )

    invoice_text = f"""
==============================
        SMART FASHION STORE
==============================
Invoice ID : {invoice_id}
Date       : {date}

Items Purchased:
{items}

------------------------------
Subtotal      : ₹{s['cart_total']}
Discount      : ₹{s['discount']}
Final Amount  : ₹{s['final_price']}
------------------------------

Thank you for shopping with us!
==============================
"""

    # Save invoice to file
    filename = f"invoice_{invoice_id}.txt"
    with open(filename, "w") as f:
        f.write(invoice_text)

    return invoice_text, filename
def post_support(s):
    if norm(s["user_input"]) == "no":
        s["response"] = "Thank you!"
        s["step"] = "end"
        return s
    s["response"] = "1) Return  2) Exchange"
    s["step"] = "return_exchange_choice"
    return s

def return_exchange_choice(s):
    items = "\n".join(f"{i+1}. {p['product']}" for i,p in enumerate(s["cart"]))
    if s["user_input"] == "1":
        s["response"] = f"Select item to return:\n{items}"
        s["step"] = "process_return"
    else:
        s["response"] = f"Select item to exchange:\n{items}"
        s["step"] = "process_exchange"
    return s

def process_return(s):
    try:
        i = int(s["user_input"]) - 1
        if 0 <= i < len(s["cart"]):
            item_name = s["cart"][i]["product"]
            # Here you would typically update inventory and handle refund logic
            s["response"] = f"Return initiated for {item_name}. Refund processed."
            s["step"] = "end"
        else:
            s["response"] = "Invalid item number. Try again."
    except ValueError:
        s["response"] = "Invalid input. Please enter a number."
    return s

def process_exchange(s):
    try:
        i = int(s["user_input"]) - 1
        if 0 <= i < len(s["cart"]):
            s["exchange_item"] = s["cart"][i]
            s["response"] = "Enter new size:"
            s["step"] = "confirm_exchange"
        else:
            s["response"] = "Invalid item number. Try again."
    except ValueError:
        s["response"] = "Invalid input. Please enter a number."
    return s

def confirm_exchange(s):
    new_size = s["user_input"].upper()
    if s["exchange_item"] and new_size in ["S", "M", "L", "XL"]:
        s["response"] = f"Exchange confirmed for {s['exchange_item']['product']}. New size {new_size} will be delivered."
        s["step"] = "end"
    else:
        s["response"] = "Invalid size. Please enter S, M, L, or XL."
    return s

# ============================================================
# CHAT LOOP
# ============================================================
NODE_MAP = {f.__name__: f for f in [
    ask_product, select_product, select_size, cart_decision,
    recommendation, shop_more, apply_offer, payment,
    support, post_support, return_exchange_choice,
    process_return, process_exchange, confirm_exchange
]}

def fashion_chatbot():
    state = {
        "user_input": "", "step": "ask_product", "category": "",
        "product": "", "size": "", "cart": [], "cart_total": 0,
        "discount": 0, "final_price": 0, "payment_attempts": 0,
        "recommended_items": [],
        "exchange_item": {}, "response": ""
    }
    print("👗 SMART FASHION STORE AI")
    while True:
        user = input("\nYou: ")
        if norm(user) == "exit":
            break

        state["user_input"] = user
        state = NODE_MAP[state["step"]](state)
        print("Bot:", state["response"])
        if state["step"] == "end":
            break

fashion_chatbot()

👗 SMART FASHION STORE AI

You: shirt
Bot: Available shirts:
Blue Formal Shirt, Black Formal Shirt, White Office Shirt

You: black formal shirt
Bot: Select size (S/M/L/XL)

You: m
Bot: Add to cart? (yes/no)

You: yes
Bot: Recommended: Navy Blue Chinos. Add? (yes/no)

You: yes
Bot: Shop more? (yes/no)

You: yes
Bot: What would you like next?

You: ethnic
Bot: Available ethnic:
Ethnic Wear 1

You: 1
Bot: Select size (S/M/L/XL)

You: l
Bot: Add to cart? (yes/no)

You: yes
Bot: Shop more? (yes/no)

You: no
Bot: 🛒 CART SUMMARY:
Black Formal Shirt – ₹1799
Navy Blue Chinos – ₹2299
Ethnic Wear 1 – ₹3499

Subtotal: ₹7597

Offers:
1) HDFC ₹300
2) ICICI ₹250
3) SBI ₹200
Choose offer:

You: 2
Bot: Final amount: ₹7347
Buy online or store?

You: online
Bot: ❌ Payment failed. Retry? (yes / no)

You: yes
Bot: ✅ Payment successful! Delivery in 3–5 days.

You: ok
Bot: Order complete. Invoice: 
        SMART FASHION STORE
Invoice ID : 27a7f59b
Date       : 17-12-2025 17:37

Items Purchased:
1. Black Forma